In [10]:
import sqlite3
import pandas as pd

# Create your connection.
from twitter.nlp_util import process_tweet

cnx = sqlite3.connect('db.sqlite3')

df = pd.read_sql_query(
    "SELECT id, conversation_id, created_at, text, author_id,in_reply_to_user_id FROM delab_timeline WHERE lang='en'",
    cnx)

df.head(3)


,id,conversation_id,created_at,text,author_id,in_reply_to_user_id
0,1,1435703745304612870,2021-09-08 20:37:01,RT @OregonOEM: 🚩🌩🔥 Red Flag and #FireWeather w...,14838508,NaN
1,2,1435664738353098756,2021-09-08 18:02:01,It's #NationalPreparednessMonth. Help ensure t...,14838508,NaN
2,3,1435663883595706370,2021-09-08 17:58:37,RT @OregonOEM: Oregon is still recovering from...,14838508,NaN


Now we are importing the libraries.

The first idea was to follow lda https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

but turns out it is to little information

In [11]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

nlp = spacy.load("en_core_web_lg")

Showing some descriptive statistics for the author corpus downloaded from twitter.

In [12]:
df.author_id.nunique()  # the number of unique authors in the dataset

69

In [13]:
df_reduced = df[["author_id", "text", "id"]]
#df_reduced = df_reduced.groupby('author_id')
# df_reduced.count()

df_reshaped = df_reduced.pivot(index="id", columns="author_id", values="text")
mask = 400 > df_reshaped.nunique()
mask = mask[mask == True]
df_reshaped.drop(columns=mask.index, inplace=True)
df_reshaped.nunique()  # the number of tweets of the authors that have more then 400 tweets

author_id
16558158               478
18616003               946
26998226               469
382814447              446
1005470991668084736    445
1106611172462219265    491
1162371171805011968    447
1239172010363826183    427
1292908140975943681    414
1402252385427222528    414
1403930956428460035    441
dtype: int64

The following takes the pandas dataframe and converts it to a dictionary with the author ids as keys and the twitter
corpora as values.

In [14]:
#df_reshaped.shape
author_corpora = df_reshaped.to_dict(orient="series")
for author_id, tweets in author_corpora.items():
    author_corpora[author_id] = tweets.dropna()

example_corpus = author_corpora[next(iter(author_corpora))]
example_corpus

id
8247                                   @ncreen_same ouch!
8248    That last tweet got some responses from spambo...
8249    'virtually' virtually means: real\n\nSo it sho...
8250                               Sounds great on paper!
8251    Any .com.au registrar recommendations? So far ...
                              ...                        
8742    RT @paydirtapp: Check out our Free Invoice Cre...
8743    @taitems @mmilo yeah nice site, suggestion: ht...
8744    RT @MichaelFHansen: Zendesk eyes Southeast Asi...
8745    While LinkedIn has been changing drastically o...
8746    Another LinkedIn email fail… now they want me ...
Name: 16558158, Length: 478, dtype: object

Now we are ready to analyze the corpora of the authors tweets. We do one example with the first corpus


In [15]:
docs = []

for tweet in example_corpus:
    docs.append(process_tweet(tweet))

# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

###############################################################################
# We remove rare words and common words based on their *document frequency*.
# Below we remove words that appear in less than 20 documents or in more than
# 50% of the documents. Consider trying to remove words only based on their
# frequency, or maybe combining that with this approach.
#

# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

###############################################################################
# Finally, we transform the documents to a vectorized form. We simply compute
# the frequency of each word, including the bigrams.
#

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

###############################################################################
# Let's see how many tokens and documents we have to train on.
#

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

# Note that we use the "Umass" topic coherence measure here (see
# :py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
# obtained an implementation of the "AKSW" topic coherence measure (see
# accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

top_topics = model.top_topics(corpus)  #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint

pprint(top_topics)

Number of unique tokens: 9
Number of documents: 478
Average topic coherence: -5.7899.
[([(0.27511057, 'work'),
   (0.25340468, 'know'),
   (0.23143622, 'look'),
   (0.22061206, 'new'),
   (0.004309774, 'like'),
   (0.0038732383, '...'),
   (0.003764999, 'use'),
   (0.0037498248, 'get'),
   (0.0037387218, '…')],
  -5.766601811687497),
 ([(0.35383725, 'like'),
   (0.34152696, '...'),
   (0.2767461, 'use'),
   (0.004997251, 'look'),
   (0.004972638, 'new'),
   (0.004517441, 'know'),
   (0.004516976, 'get'),
   (0.004463332, 'work'),
   (0.0044220276, '…')],
  -5.784562450219087),
 ([(0.49528044, '…'),
   (0.45598957, 'get'),
   (0.0073577045, 'work'),
   (0.007170174, 'know'),
   (0.007053388, 'like'),
   (0.0068173804, '...'),
   (0.0068133674, 'look'),
   (0.006772335, 'use'),
   (0.006745627, 'new')],
  -5.818532690537116)]


As this resulted in disappointing non-nouns as topics the alternative would be to use the named entities directly


In [16]:
found_entities = set()

for tweet in example_corpus:
    doc = nlp(tweet)
    sentences = list(doc.sents)
    for sentence in sentences:
        if sentence.ents:
            found_entities.update(sentence.ents)

entities = list(found_entities)[1:10]
entities


[@rioter @alandownie @ncreen_same,
 @alandownie,
 11000 hours,
 iOS,
 Zendesk,
 #Melbourne #,
 tomorrow,
 10k,
 a few seconds]

The next steps would be
- clean the NERs (one word), no numbers, no urls, no hashtags...
- get the embeddings for the "regular" words

In [17]:
def clean_entities(entities):
    result = []
    dropped = []
    for entity in entities:
        entity = entity.text
        if "@" in entity or "RT" in entity or entity.isnumeric() or "http" in entity or "www" in entity:
            dropped.append(entity)
        else:
            result.append(entity)
    return result, dropped


result, dropped = clean_entities(found_entities)

print("kept {}".format(result[1:50]))
print("dropped {}".format(dropped[1:50]))


kept ['iOS', 'Zendesk', '#Melbourne #', 'tomorrow', '10k', 'a few seconds', 'Lucasfilm', 'One Million', 'Rebooting Windows', 'Australia', 'SAB Miller', 'Melbourne', 'Alfa Beer', 'IPA', 'LinkedIn', '28th Mar 2013', '#', 'LinkedIn', '7pm', 'Game Changer', 'americans', 'Melbourne', 'Zendesk', 'NFP', "Bottle Share'", 'five', 'SQL', 'today', 'Melbourne', 'SAB Miller', 'Athenian Brewery', 'Carlsberg Group', 'this year', 'one', 'PS4', '25%', 'WebEx', 'Disney', 'Phil Schiller', 'one', 'Budapest', 'Los Angeles', 'bush', 'the next week', 'BugHerd', 'Heineken International', 'tomorrow', 'Heineken', 'the half million']
dropped ['@rioter @alandownie @ncreen_same', '@alandownie', 'RT @merryuxmas:', '@buzzswarbrick', '@alandownie @mmilo', '3124', 'RT @MichaelFHansen', '@nikiscevak', '@mmilo', '@rioter', 'http://t.co/bB7Uzzqo', '@alandownie @ncreen_same', '@gartner', 'RT @paydirtapp:', 'RT', '234272', '@alandownie @vbrendel', '@medhved', '@alandownie @mmilo', 'http://t.co/qsvud4CHF6', 'RT @veroapp', '

In the following we are using fasttext to get vector representations of the named entities

_@inproceedings{grave2018learning,
  title={Learning Word Vectors for 157 Languages},
  author={Grave, Edouard and Bojanowski, Piotr and Gupta, Prakhar and Joulin, Armand and Mikolov, Tomas},
  booktitle={Proceedings of the International Conference on Language Resources and Evaluation (LREC 2018)},
  year={2018}
}_

**Careful! Loading the fasttext model into memory requires a lot of memory and time**

For this reason, the code is commented out.

In [18]:
import fasttext.util
# ft = fasttext.load_model('/home/julian/nltk_data/fasttext/cc.de.300.bin')
# word_vec_example = ft.get_word_vector('König')
# word_vec_example[1:10]

from stackoverflow the following idea to store the embeddings in the db

```python
from django.db import models
np_field = models.BinaryField()
# transform numpy array to python byte using pickle dumps, then encoded by base64
np_bytes = pickle.dumps(np_array)
np_base64 = base64.b64encode(np_bytes)
model.np_field = np_base64
get the numpy array from django model
np_bytes = base64.b64decode(model.np_field)
np_array = pickle.loads(np_bytes)
```
